In [77]:
import logging
import pathlib
import re

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import geojson


In [76]:
filename = '~/Downloads/reanalysis-surge-1979-m01.nc'
filename = '/Users/baart_f/Downloads/reanalysis-tide-1979-m01.nc'


def export_geojson(ds):
    features = []
    for x, y, name in zip(ds.station_x_coordinate.values, ds.station_y_coordinate.values, ds.station_name.values):
        features.append(
            geojson.Feature(id=name, geometry=geojson.Point([x, y]))
        )
    result = geojson.FeatureCollection(features)

    with open('gtsm-crs.json', 'w') as f:
        geojson.dump(result, f)

In [80]:
def gtsm2mean(filename):
    """gtsm"""
    path = pathlib.Path(filename).expanduser()
    
    pattern = re.compile(
        r'reanalysis-(?P<variable>\w+)-(?P<year>\d+)-m(?P<month>\d+)\.nc'
    )
    match = pattern.search(filename)
    if not match:
        raise ValueError('filenmae not recognized')
    variable = match.group('variable')
    
    with xr.open_dataset(filename) as ds:
        ids = [
            b"NWS_NO_TS_MO_Vlissingen",
            b"id_obs_hoekvanholla_hvh_nl_rws",
            b"NWS_NO_TS_MO_Ijmuiden",
            b"id_obs_denhelder_hel_nl_rws",
            b"NWS_NO_TS_MO_Harlingen",
            b"id_obs_delfzijl_del_nl_rws"
        ]
        idx = ds.station_name.isin(ids)
        station_names = ds.station_name[idx][:]
        columns = [x.decode() for x in station_names.values]
        df = ds[variable][:, idx].to_pandas()
        df.columns = columns
        record = df.mean()
        record['time'] = df.index.mean()
        record['filename'] = filename
        record['variable'] = variable
        
    return record

%timeit gtsm2mean(filename)

377 ms ± 28.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
